In [1]:
# file load
#%matplotlib inline
%matplotlib
import pandas as pd 
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

train = pd.read_csv("raw/train.csv") # 136*43 matrix
test = pd.read_csv("raw/test.csv") # 99999*42 matrix

Using matplotlib backend: MacOSX


In [2]:
%matplotlib
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=1, ncols=3)
m = axes.flat
X = pd.Series([1,2,3])
Y = pd.Series([0.1,0.2,0.3])

i=0
for name in ["sums","log_mul","avg"]:
    m[i].set_title(name)
    m[i].scatter(X,Y, alpha=0.5)
    i+=1

plt.show()

Using matplotlib backend: MacOSX


In [2]:
# convert date to days
import time
from datetime import datetime as dt
# train
all_diff = []
for item in train["Open Date"]:
    diff = dt.now() - dt.strptime(item, "%m/%d/%Y")
    all_diff.append(int(diff.days/365))

train['DaysOfRest'] = pd.Series(all_diff)

# test
all_diff = []
for item in test["Open Date"]:
    diff = dt.now() - dt.strptime(item, "%m/%d/%Y")
    all_diff.append(int(diff.days/365))

test['DaysOfRest'] = pd.Series(all_diff)
print train.shape, test.shape

(137, 44) (100000, 43)


In [3]:
# drop id and open date
useless_features = ['Id','Open Date','City Group']
train = train.drop(useless_features, 1)
test = test.drop(useless_features, 1)
print train.keys()
print test.keys()

Index([u'City', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'revenue', u'DaysOfRest'], dtype='object')
Index([u'City', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'DaysOfRest'], dtype='object')


In [4]:
# add new field = max value
train_normal = train.copy()

names = [name for name in train.keys() if name[0]=='P']
max_params = []
j = 0
for i in range(0,len(train_normal)):
    max_value = 0
    for name in names:
        if train_normal.iloc[i][name] > max_value:
            max_value = train_normal.iloc[i][name]
    if max_value > 5:
        j +=1
    #print max_value
    max_params.append(max_value)
    
train_normal['max_params'] = pd.Series(max_params)
print train_normal.keys()
print train_normal['max_params'].value_counts()
print train_normal.shape

Index([u'City', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'revenue', u'DaysOfRest', u'max_params'], dtype='object')
5     114
25     12
15      5
10      2
20      2
18      2
dtype: int64
(137, 42)


In [5]:
# convert strange data to good
names = [name for name in train_normal.keys() if name[0]=='P']
multiplier = {'P1':3,'P2':1.5,'P3':1.5,'P4':1.5,'P5':2,'P6':2,'P7':2,'P8':2,'P9':2,'P10':2,'P11':2,'P12':2,'P13':1.5,'P14':3,
              'P15':2,'P16':3,'P17':3,'P18':3,'P19':5,'P20':3,'P21':3,'P22':1,'P23':5,'P24':2,'P25':2,'P26':2.5,'P27':2.5,
              'P28':2.5,'P29':2.5,'P30':5,'P31':3,'P32':5,'P33':2,'P34':6,'P35':3,'P36':4,'P37':2}

for i in range(0,len(train_normal)):
    if train_normal.iloc[i]['max_params'] > 5:
        for name in names:
            #train[item] = train[item].astype('category')
            train_normal[name]=train_normal[name].astype(np.float64)
            train_normal.loc[i,name] /= multiplier[name]
    

In [6]:
# normalize test arguments
test_normal = test.copy()
for i in range(0,len(test)):
    for name in names:
        val = test_normal.iloc[i][name]
        if val % 1 != 0 or val > 5:
            test_normal.loc[i,name] /= multiplier[name]
        if name == 'P7' and val == 2: # probably useless
            test_normal.loc[i,name] /= 2
        if name == 'P29' and val == 5:
            test_normal.loc[i,name] /= 2.5
    if i % 5000 == 0:
        print i
print "End"

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
End


In [7]:
# replace low freq city
big_city = ['İstanbul', 'Ankara', 'İzmir']
for i in range(0,len(train_normal)):
    if train_normal.iloc[i]["City"] not in big_city:
        train_normal.loc[i,"City"] = 'Other'

for i in range(0,len(test_normal)):
    if test_normal.iloc[i]["City"] not in big_city:
        test_normal.loc[i,"City"] = 'Other'    

In [26]:
for name in test_normal.keys():
    print name
    print dict(train_normal[name].value_counts())
    print dict(test_normal[name].value_counts())
    print set(test_normal[name].value_counts().keys()) - set(train_normal[name].value_counts().keys())

# Type: DT, MB = NA
# P3: 0 = NA
# P4: 2 = NA
# P5: 5 = NA
# P7: 3,4 = SAME = RARE
# P8: 1,2 = NA
# P9: 3 = NA
# P13: 3 = NA
# P29: 0 = NA

# VERY GOOD VARIABLES: City, Days, P1, P2, P6, P10, P11, P12, P19, P20, P21, P22, P23, P28
# GOOD VARIABLES: P3, P4, P5, P7, P8, P9, P13, P29, Type

City
{'Ankara': 19, '\xc4\xb0zmir': 9, 'Other': 59, '\xc4\xb0stanbul': 50}
{'Ankara': 8720, '\xc4\xb0zmir': 6465, 'Other': 50728, '\xc4\xb0stanbul': 34087}
set([])
Type
{'DT': 1, 'FC': 76, 'IL': 60}
{'DT': 2244, 'FC': 57019, 'IL': 40447, 'MB': 290}
set(['MB'])
P1
{1.0: 10, 2.0: 42, 3.0: 34, 4.0: 47, 5.0: 4}
{1: 5197, 2: 26562, 3: 25335, 4: 35147, 5: 7759}
set([])
P2
{1.0: 5, 2.0: 13, 3.0: 16, 4.0: 39, 5.0: 64}
{1.0: 1503, 2.0: 8719, 3.0: 17052, 4.0: 23009, 5.0: 49717}
set([])
P3
{0.0: 1, 2.0: 3, 3.0: 11, 4.0: 104, 5.0: 18}
{0.0: 318, 2.0: 1906, 3.0: 9959, 4.0: 77983, 5.0: 9834}
set([])
P4
{3.0: 22, 4.0: 89, 5.0: 26}
{2.0: 311, 3.0: 12376, 4.0: 66497, 5.0: 20816}
set([2.0])
P5
{1.0: 59, 2.0: 65, 3.0: 7, 4.0: 5, 5.0: 1}
{1: 33587, 2: 50633, 3: 5931, 4: 9562, 5: 287}
set([])
P6
{1.0: 12, 2.0: 61, 3.0: 28, 4.0: 17, 5.0: 19}
{1: 9850, 2: 43925, 3: 20554, 4: 16873, 5: 8798}
set([])
P7
{1.0: 12, 3.0: 1, 4.0: 3, 5.0: 121}
{1: 9109, 3: 1864, 4: 1884, 5: 87143}
set([])
P8
{1.0: 1, 2.0: 1, 3.0: 7

In [8]:
# make sneaky NA fill
medium_full = {'P7':[3,4]}
na_fill = {'Type':['DT','MB'], 'P4':[2],'P5':[5],'P8':[1,2],'P9':[3], 'P13':[3],'P29':[4]}

# na fill train arguments
train_normal_na = train_normal.copy()
for i in range(0,len(train_normal_na)):
    for name in train_normal_na.keys():
        val = train_normal_na.iloc[i][name]
        if val == 0:
            train_normal_na.loc[i,name] = np.NAN
        elif name in medium_full.keys():
            if val in medium_full[name]:
                train_normal_na.loc[i,name] = np.average(medium_full[name]) 
        elif name in na_fill:
            if val in na_fill[name]:
                train_normal_na.loc[i,name] = np.NAN


In [9]:
# na fill test arguments
test_normal_na = test_normal.copy()
for i in range(0,len(test_normal_na)):
    for name in test_normal_na.keys():
        val = test_normal_na.iloc[i][name]
        if val == 0:
            test_normal_na.loc[i,name] = np.NAN
        elif name in medium_full.keys():
            if val in medium_full[name]:
                test_normal_na.loc[i,name] = np.average(medium_full[name]) 
        elif name in na_fill:
            if val in na_fill[name]:
                test_normal_na.loc[i,name] = np.NAN

    if i % 5000 == 0:
        print i
print "End"

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
End


In [107]:
import random
# try to impute values
good_features = ['City', 'DaysOfRest', 'P1', 'P2', 'P6','P7', 'P10', 'P11', 'P12', 'P19', 'P20', 'P21', 'P22', 'P23', 'P28']
normal_features = ['P3', 'P4', 'P5', 'P8', 'P9', 'P13', 'P29', 'Type']

# fill type with mean!!!
for i in train_normal_na.index:
    val = train_normal_na.iloc[i]['Type']
    if val == 'FC':
        train_normal_na.loc[i,'Type'] = np.int64(1)
    elif val == 'IL':
        train_normal_na.loc[i,'Type'] = np.int64(2)

for i in test_normal_na.index:
    val = test_normal_na.iloc[i]['Type']
    if val == 'FC':
        test_normal_na.loc[i,'Type'] = np.int64(1)
    elif val == 'IL':
        test_normal_na.loc[i,'Type'] = np.int64(2)

all_samples['Type'] = all_samples['Type'].astype(np.float64)

all_samples = train_normal_na.copy()
all_samples = all_samples.drop(['revenue','max_params'],1)
all_samples = pd.concat([all_samples,test_normal_na])
good_train = all_samples.loc[:,good_features]
good_train = pd.get_dummies(good_train)



In [66]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from random import sample

for name in normal_features:#
    print name
    print "***********************"
    imp_value = all_samples[name]
    test_imp = imp_value[pd.isnull(imp_value)]
    train_imp = imp_value[pd.notnull(imp_value)]
    # make baselines
    vals = train_imp.value_counts()
    top_result = pd.Series(np.repeat(vals.keys()[0],len(train_imp)))
    random_result = pd.Series([vals.keys()[random.randint(0,len(vals.keys())-1)] for i in range(0,len(train_imp))])
    top_result = top_result.reset_index(drop=True)
    random_result = random_result.reset_index(drop=True)
    
    # get train things 
    X = good_train.iloc[train_imp.index]
    Y = train_imp
    X = X.reset_index(drop=True)
    Y = Y.reset_index(drop=True)
    X[str(name+"_mean")] = top_result.reset_index(drop=True)
    X_pred = good_train.iloc[test_imp.index]
    Y = np.array(Y).astype(int) # fix FFSH problems
    
    # Create classifiers
    params = {'n_estimators':100, 'max_features':None, 'max_depth':6}
    rf = RandomForestClassifier(**params)
    ada = AdaBoostClassifier(n_estimators=100)
    knn = KNeighborsClassifier(n_neighbors=10, weights='distance')
    knn2 = KNeighborsClassifier(n_neighbors=100, weights='uniform')
    sgd = SGDClassifier(loss = 'log', alpha=0.01, n_iter=200)
    all_class = [(rf, 'RF'),
                (ada, 'Ada'),
                (knn, 'KNN'),
                (knn2,'KNN2'),
                (sgd, 'SGD')]

    # initial model
    # multi_class =  OneVsRestClassifier(KNeighborsClassifier(n_neighbors=10, weights='distance'))
    X_extended = X.copy()
    kf2 = cross_validation.KFold(len(X), n_folds=2,shuffle=True)
    for train_index, test_index in kf2: 
        avg_acc = []
        for multi_class, cl_name in all_class:
            print "Train: ", cl_name
            multi_class.fit(X.iloc[train_index], Y[train_index])
            X_extended[cl_name] = multi_class.predict(X)
            acc = accuracy_score(Y[test_index], X_extended[cl_name].iloc[test_index])
            print acc
        print X_extended.shape
    
    kf = cross_validation.KFold(len(X), n_folds=10,shuffle=True)
    # final 
    multi_class =  OneVsRestClassifier(SGDClassifier(loss = 'log', alpha=0.01, n_iter=200))
    #multi_class =  OneVsRestClassifier(RandomForestClassifier(**params))
    for train_index, test_index in kf:        
        multi_class.fit(X_extended.iloc[train_index], Y[train_index])
        pred = multi_class.predict(X_extended.iloc[test_index])
        acc = accuracy_score(Y[test_index], pred)
        avg_acc.append(acc)
        print("Acc: %.2f" % acc)

    print("AVG ACC: %.2f MAX ACC: %.2f MIN ACC: %.2f" % (np.mean(avg_acc), np.max(avg_acc), np.min(avg_acc)))
    print(("RANDOM ACC: %.2f MODE ACC: %.2f" % (accuracy_score(Y,random_result), accuracy_score(Y,top_result))))
    multi_class.fit(X, Y)
    res = multi_class.predict(X)


P3
***********************
Train:  RF
0.780260073333
Train:  Ada
0.780260073333
Train:  KNN
0.768959506301
Train:  KNN2
0.780260073333
Train:  SGD
0.780260073333
(99818, 24)
Train:  RF
0.784327476006
Train:  Ada
0.784327476006
Train:  KNN
0.773567893566
Train:  KNN2
0.784327476006
Train:  SGD
0.784327476006
(99818, 24)
Acc: 0.87
Acc: 0.87
Acc: 0.87
Acc: 0.87
Acc: 0.87
Acc: 0.87
Acc: 0.87
Acc: 0.87
Acc: 0.87
Acc: 0.87
AVG ACC: 0.87 MAX ACC: 0.87 MIN ACC: 0.87
RANDOM ACC: 0.25 MODE ACC: 0.78
P4
***********************
Train:  RF
0.668563300142
Train:  Ada
0.668563300142
Train:  KNN
0.633602468295
Train:  KNN2
0.668563300142
Train:  SGD
0.668563300142
(99826, 24)
Train:  RF
0.665477931601
Train:  Ada
0.665477931601
Train:  KNN
0.628794101737
Train:  KNN2
0.665477931601
Train:  SGD
0.665477931601
(99826, 24)
Acc: 0.79
Acc: 0.79
Acc: 0.79
Acc: 0.79
Acc: 0.79
Acc: 0.79
Acc: 0.80
Acc: 0.79
Acc: 0.79
Acc: 0.80
AVG ACC: 0.79 MAX ACC: 0.80 MIN ACC: 0.79
RANDOM ACC: 0.33 MODE ACC: 0.67
P5
*******

KeyboardInterrupt: 

In [127]:
# normal values imputation

from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from random import sample

train_normal_imputed = train_normal_imputed.copy()#train_normal_na.copy()
test_normal_imputed = test_normal_imputed.copy()#test_normal_na.copy()

for name in normal_features:
    print name
    print "***********************"
    # make train set
    imp_value = all_samples[name]
    train_imp = imp_value[pd.notnull(imp_value)]
    
    # make baselines
    vals = train_imp.value_counts()
    top_result = pd.Series(np.repeat(vals.keys()[0],len(train_imp)))
    random_result = pd.Series([vals.keys()[random.randint(0,len(vals.keys())-1)] for i in range(0,len(train_imp))])
    top_result = top_result.reset_index(drop=True)
    random_result = random_result.reset_index(drop=True)
    
    # get train things 
    X = good_train.iloc[train_imp.index]
    Y = train_imp
    X = X.reset_index(drop=True)
    Y = Y.reset_index(drop=True)
    
    # add mean expectation
    X[str(name+"_mean")] = top_result.reset_index(drop=True)
    X_pred = good_train.iloc[test_imp.index]
    Y = np.array(Y).astype(int) # fix FFSH problems
    
    # Create classifiers
    params = {'n_estimators':100, 'max_features':None, 'max_depth':6}
    rf = RandomForestClassifier(**params)
    ada = AdaBoostClassifier(n_estimators=100)
    knn = KNeighborsClassifier(n_neighbors=10, weights='distance')
    knn2 = KNeighborsClassifier(n_neighbors=100, weights='uniform')
    sgd = SGDClassifier(loss = 'log', alpha=0.01, n_iter=200)
    all_class = [(rf, 'RF'),
                (ada, 'Ada'),
                (knn, 'KNN'),
                (knn2,'KNN2'),
                (sgd, 'SGD')]

    # initial models 
    X_extended = X.copy()
    
    for multi_class, cl_name in all_class:
        print "Train: ", cl_name
        multi_class.fit(X, Y)
        X_extended[cl_name] = multi_class.predict(X)
        print "Extended with ",cl_name," shape ",X_extended.shape
    
    # consolidated model  
    print "Train consolidated model"
    cons_class =  OneVsRestClassifier(SGDClassifier(loss = 'log', alpha=0.01, n_iter=200))       
    cons_class.fit(X_extended, Y)
    
    # fill imputed values
    for df_to_fill in (train_normal_imputed,test_normal_imputed):
        accum_df = df_to_fill.copy()
        if 'revenue' in accum_df.keys():
            accum_df = accum_df.drop(['revenue','max_params'],1)
        goal_col = accum_df[name]
        goal_indexes = goal_col[pd.isnull(goal_col)].index
        
        # if nothing to impute
        if len(goal_indexes) == 0:
            print "Nothing to impute. Skip this imputation"
            continue
        
        accum_df = accum_df.loc[goal_indexes,good_features]
        accum_df = pd.get_dummies(accum_df)
        # add mean
        accum_df[str(name+"_mean")] = pd.Series(np.repeat(vals.keys()[0],len(accum_df)), index=goal_indexes)
        
        # fill missed keys
        for null_name in set(X.keys())-set(accum_df.keys()):
            accum_df[null_name] = pd.Series(np.repeat(0,len(accum_df)), index=goal_indexes)
        
        print "To predict shape: ",accum_df.shape, " Train shape ", X.shape
        
        base_accum_df = accum_df.copy()
        for multi_class, cl_name in all_class:
            print "Predict: ", cl_name
            accum_df[cl_name] = multi_class.predict(base_accum_df)
            print dict(accum_df[cl_name].value_counts(dropna=False))
            print "Extended with ",cl_name," shape ",accum_df.shape
        
        print "Predict consolidated"
        
        # predict consolidated
        df_to_fill.loc[goal_indexes, name] = cons_class.predict(accum_df)
        print dict(pd.Series(cons_class.predict(accum_df)).value_counts())
        
        print "Check for imputation"
        print dict(df_to_fill[name].value_counts(dropna=False))



P14
***********************
Train:  RF
Extended with  RF  shape  (34315, 29)
Train:  Ada
Extended with  Ada  shape  (34315, 30)
Train:  KNN
Extended with  KNN  shape  (34315, 31)
Train:  KNN2
Extended with  KNN2  shape  (34315, 32)
Train:  SGD
Extended with  SGD  shape  (34315, 33)
Train consolidated model
To predict shape:  (88, 28)  Train shape  (34315, 28)
Predict:  RF
{3: 88}
Extended with  RF  shape  (88, 29)
Predict:  Ada
{3: 88}
Extended with  Ada  shape  (88, 30)
Predict:  KNN
{1: 8, 2: 8, 3: 59, 4: 8, 5: 5}
Extended with  KNN  shape  (88, 31)
Predict:  KNN2
{3: 88}
Extended with  KNN2  shape  (88, 32)
Predict:  SGD
{3: 88}
Extended with  SGD  shape  (88, 33)
Predict consolidated
{1: 8, 3: 75, 5: 5}
Check for imputation
{nan: 0, 1.0: 20, 2.0: 10, 3.0: 90, 4.0: 7, 5.0: 10}
To predict shape:  (65734, 28)  Train shape  (34315, 28)
Predict:  RF
{3: 65734}
Extended with  RF  shape  (65734, 29)
Predict:  Ada
{3: 65734}
Extended with  Ada  shape  (65734, 30)
Predict:  KNN
{1: 6309, 2:

In [126]:
# return type
# fill type with mean!!!
for i in train_normal_imputed.index:
    val = train_normal_imputed.iloc[i]['Type']
    if val == 1:
        train_normal_imputed.loc[i,'Type'] = 'FC'
    elif val == 2:
        train_normal_imputed.loc[i,'Type'] = 'IL'

for i in test_normal_imputed.index:
    val = test_normal_imputed.iloc[i]['Type']
    if val == 1:
        test_normal_imputed.loc[i,'Type'] = 'FC'
    elif val == 2:
        test_normal_imputed.loc[i,'Type'] = 'IL'
        
good_features = ['City', 'DaysOfRest', 'P1', 'P2', 'P6','P7', 'P10', 'P11', 'P12', 'P19', 'P20', 'P21', 'P22', 'P23', 'P28',
                'P3', 'P4', 'P5', 'P8', 'P9', 'P13', 'P29', 'Type']

normal_features = ['P14','P15','P16','P17','P18','P24','P25','P26','P27','P30','P31','P32','P33','P34','P35','P36','P37']


all_samples = train_normal_imputed.copy()
all_samples = all_samples.drop(['revenue','max_params'],1)
all_samples = pd.concat([all_samples,test_normal_imputed])
good_train = all_samples.loc[:,good_features]
good_train = pd.get_dummies(good_train)

In [128]:
for name in test_normal_imputed.keys():
    print name
    print test_normal_imputed[name].value_counts(dropna=False)

City
Other       50728
İstanbul    34087
Ankara       8720
İzmir        6465
NaN             0
dtype: int64
Type
FC     58576
IL     41424
NaN        0
dtype: int64
P1
4    35147
2    26562
3    25335
5     7759
1     5197
dtype: int64
P2
 5     49717
 4     23009
 3     17052
 2      8719
 1      1503
NaN        0
dtype: int64
P3
 4     78298
 3      9962
 5      9834
 2      1906
NaN        0
dtype: int64
P4
 4     66761
 5     20863
 3     12376
NaN        0
dtype: int64
P5
 2     50799
 1     33685
 4      9585
 3      5931
NaN        0
dtype: int64
P6
2    43925
3    20554
4    16873
1     9850
5     8798
dtype: int64
P7
 5.0    87143
 1.0     9109
 3.5     3748
NaN         0
dtype: int64
P8
 5     46280
 4     45953
 3      7767
NaN        0
dtype: int64
P9
 5     58192
 4     41808
NaN        0
dtype: int64
P10
5    74791
4    25209
dtype: int64
P11
3    24118
2    23079
5    17834
1    17781
4    17188
dtype: int64
P12
5    57715
4    29330
3    11680
2     1275
dtype: int64
P1

In [130]:
test_normal_imputed.to_csv("raw/test_imputed_final.csv", index=False)
train_normal_imputed.to_csv("raw/train_imputed_final.csv", index=False)

In [24]:
#3    48224
#4    10453
#5     7198

names = [name for name in train_normal.keys() if name[0]=='P']
names = [name for name in names if 0 not in train_normal[name].value_counts().keys()]
print names 
i_range = [124]
for i in i_range:
    lowest_sum = 1000
    lowest_j = 0
    for j in range(0, len(test_normal)):
        cum_sum = 0
        for name in names:
            cum_sum += np.abs(train_normal.iloc[i][name]- test_normal.iloc[j][name])
        #print outlier_index[i], j,  cum_sum
        if j != i:
            if cum_sum <=lowest_sum:
                lowest_sum = cum_sum
                lowest_j = j
    print "**************"
    print "Nearest item"
    print i, lowest_sum, lowest_j#, np.abs(train_normal.iloc[i]["revenue"] - test_normal.iloc[lowest_j]["revenue"])

['P1', 'P2', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P19', 'P20', 'P21', 'P22', 'P23', 'P28']
**************
Nearest item
124 6.0 7971


In [9]:
# remove outlier from train set
outlier_index = [16]
train_normal = train_normal.drop(train_normal.index[outlier_index])
train_normal = train_normal.reset_index(drop=True)
print len(train_normal)

136


In [13]:
# pca
from sklearn.decomposition import PCA,RandomizedPCA,SparsePCA,KernelPCA
X = train_normal[names]
pca = PCA()
pca.fit(X)
X_trunc = pca.transform(X)

In [11]:
# pca bad
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_trunc[:,0], X_trunc[:,1], X_trunc[:,2], c="r", marker="o")

In [20]:
# bad
print pca.explained_variance_ratio_
for i in range(0,len(names)):
    print names[i], pca.components_[0][i]
print len(pca.components_[0])

[  6.12307643e-01   1.18907668e-01   4.05723771e-02   3.47032391e-02
   2.28754460e-02   2.02665245e-02   1.76767211e-02   1.58452423e-02
   1.27161335e-02   1.25245689e-02   1.00860668e-02   8.18358530e-03
   6.92791780e-03   6.83357080e-03   6.41567666e-03   5.77708486e-03
   5.38459804e-03   5.15340508e-03   4.12665585e-03   3.89116256e-03
   3.66848139e-03   3.47316340e-03   3.19864798e-03   2.68010673e-03
   2.29649553e-03   2.12471936e-03   1.85004970e-03   1.72751444e-03
   1.55272717e-03   1.38742332e-03   1.30052007e-03   1.11961983e-03
   1.01678643e-03   5.45154424e-04   4.92593963e-04   3.31044341e-04
   5.96645016e-05]
P1 -0.0363138782099
P2 -0.0335208812681
P3 -0.0160455835125
P4 0.0130171232
P5 0.0186945273537
P6 -0.0441421489887
P7 -0.0326618763872
P8 0.0102433025588
P9 -0.00221416121377
P10 0.000701553813505
P11 -0.0156016488259
P12 0.0024764563193
P13 -0.00111598965204
P14 -0.204089783331
P15 -0.233547320079
P16 -0.264999498973
P17 -0.169321958091
P18 -0.269058519012


In [62]:
# split test by 2 similar to train groups
zero_keys = []
# select zero names
for name in names:
    if 0 in test_normal[name].value_counts().keys():
        if name not in ['P3','P29']:
            zero_keys.append(name)

not_null_test = test_normal.copy()
null_test = test_normal.copy()

for name in zero_keys:
    print "Erase zero key: ", name
    if 0 in test_normal[name].value_counts().keys():
        not_null_test = not_null_test[not_null_test[name] != 0]
        null_test = null_test[null_test[name]==0]
    print "Erase zero key: ", name, " Items left: ", len(not_null_test), len(null_test)



Erase zero key:  P14
Erase zero key:  P14  Items left:  34266 65734
Erase zero key:  P15
Erase zero key:  P15  Items left:  24145 55651
Erase zero key:  P16
Erase zero key:  P16  Items left:  19154 50389
Erase zero key:  P17
Erase zero key:  P17  Items left:  16445 46997
Erase zero key:  P18
Erase zero key:  P18  Items left:  14517 44653
Erase zero key:  P24
Erase zero key:  P24  Items left:  13173 42876
Erase zero key:  P25
Erase zero key:  P25  Items left:  12170 41462
Erase zero key:  P26
Erase zero key:  P26  Items left:  11410 40305
Erase zero key:  P27
Erase zero key:  P27  Items left:  10762 39342
Erase zero key:  P30
Erase zero key:  P30  Items left:  10220 38492
Erase zero key:  P31
Erase zero key:  P31  Items left:  9795 37808
Erase zero key:  P32
Erase zero key:  P32  Items left:  9407 37209
Erase zero key:  P33
Erase zero key:  P33  Items left:  9073 36676
Erase zero key:  P34
Erase zero key:  P34  Items left:  8735 36192
Erase zero key:  P35
Erase zero key:  P35  Items lef

# Research correlation

In [37]:
# corr gramm test
import seaborn as sns
X_pure = test_normal[names].values

sns.set(style="darkgrid")

f, ax = plt.subplots(figsize=(9, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.corrplot(X_pure, annot=False, sig_stars=False,
             diag_names=False, cmap=cmap, ax=ax)


In [38]:
# corr gramm train
import seaborn as sns
no35 = train_normal[train_normal['P35']==0]
yes35 = train_normal[train_normal['P35']!=0]
X_pure = yes35[names].values


sns.set(style="darkgrid")

f, ax = plt.subplots(figsize=(9, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.corrplot(X_pure, annot=False, sig_stars=False,
             diag_names=False, cmap=cmap, ax=ax)

In [69]:
# correlated items
corr_matrix = no35[names].corr()
print "Not empty lines"
for name_x in corr_matrix.keys():
    for name_y in corr_matrix.keys():
        item = corr_matrix[name_x][name_y]
        if name_x != name_y and np.abs(item) > 0.8:
            print name_x, name_y, item

corr_matrix = yes35[names].corr()
print "Empty lines"
for name_x in corr_matrix.keys():
    for name_y in corr_matrix.keys():
        item = corr_matrix[name_x][name_y]
        if name_x != name_y and np.abs(item) > 0.8:
            print name_x, name_y, item
            
corr_matrix = not_null_test[names].corr()
print "Empty lines"
for name_x in corr_matrix.keys():
    for name_y in corr_matrix.keys():
        item = corr_matrix[name_x][name_y]
        if name_x != name_y and np.abs(item) > 0.5:
            print name_x, name_y, item

corr_matrix = null_test[names].corr()
print "Empty lines"
for name_x in corr_matrix.keys():
    for name_y in corr_matrix.keys():
        item = corr_matrix[name_x][name_y]
        if name_x != name_y and np.abs(item) > 0.5:
            print name_x, name_y, item

Not empty lines
P1 P2 0.818096276914
P2 P1 0.818096276914
P8 P11 -0.846535557868
P9 P12 0.874144223297
P10 P13 0.959005895546
P11 P8 -0.846535557868
P12 P9 0.874144223297
P13 P10 0.959005895546
Empty lines
P1 P2 0.822178969605
P2 P1 0.822178969605
P8 P11 -0.825490412367
P9 P12 0.8717022472
P10 P13 1.0
P11 P8 -0.825490412367
P12 P9 0.8717022472
P13 P10 1.0
P24 P26 0.841740750669
P26 P24 0.841740750669
Empty lines
P1 P2 0.51927908834
P2 P1 0.51927908834
P8 P11 -0.515178514879
P9 P12 0.593583794029
P10 P13 0.634823618865
P11 P8 -0.515178514879
P12 P9 0.593583794029
P13 P10 0.634823618865
P24 P26 0.513538348788
P26 P24 0.513538348788
Empty lines
P9 P12 0.53218967288
P10 P13 0.566411618733
P12 P9 0.53218967288
P13 P10 0.566411618733
{2050: 58, 5635: 2, 2564: 1, 1032: 2, 1545: 3, 2060: 83, 2578: 81, 1555: 3, 3098: 73, 1563: 87, 543: 1, 5664: 69, 688: 1, 2082: 2, 1059: 82, 548: 2, 7207: 65, 2089: 1, 2712: 1, 6187: 70, 1580: 60, 559: 2, 1584: 83, 4145: 73, 563: 1, 1076: 71, 1590: 1, 1591: 3, 6

In [75]:
# as dummies
train_categorial = no35.copy()
test_categorial = null_test.copy()
for item in names:
    train_categorial[item] = train_categorial[item].astype(np.int64)
    train_categorial[item] = train_categorial[item].astype('category')
    test_categorial[item] = test_categorial[item].astype(np.int64)
    test_categorial[item] = test_categorial[item].astype('category')

train_categorial = pd.get_dummies(train_categorial)
train_categorial = train_categorial.drop('revenue', 1)
train_categorial = train_categorial.drop('max_params', 1)
test_categorial = pd.get_dummies(test_categorial)
# corr

print train_categorial.keys()

corr_matrix = train_categorial.corr()
print "Empty lines"
for name_x in corr_matrix.keys():
    for name_y in corr_matrix.keys():
        item = corr_matrix[name_x][name_y]
        if name_x != name_y and np.abs(item) > 0.8:
            print name_x, name_y, item
            
corr_matrix = test_categorial.corr()
print "Empty lines"
for name_x in corr_matrix.keys():
    for name_y in corr_matrix.keys():
        item = corr_matrix[name_x][name_y]
        if name_x != name_y and np.abs(item) > 0.8:
            print name_x, name_y, item

Index([u'DaysOfRest', u'City_Ankara', u'City_Other', u'City_İstanbul', u'City_İzmir', u'City Group_Big Cities', u'City Group_Other', u'Type_DT', u'Type_FC', u'Type_IL', u'P1_1', u'P1_2', u'P1_3', u'P1_4', u'P1_5', u'P2_1', u'P2_2', u'P2_3', u'P2_4', u'P2_5', u'P3_0', u'P3_2', u'P3_3', u'P3_4', u'P3_5', u'P4_3', u'P4_4', u'P4_5', u'P5_1', u'P5_2', u'P5_3', u'P5_4', u'P5_5', u'P6_1', u'P6_2', u'P6_3', u'P6_4', u'P6_5', u'P7_1', u'P7_3', u'P7_4', u'P7_5', u'P8_2', u'P8_3', u'P8_4', u'P8_5', u'P9_4', u'P9_5', u'P10_4', u'P10_5', u'P11_1', u'P11_2', u'P11_3', u'P11_4', u'P11_5', u'P12_2', u'P12_3', u'P12_4', u'P12_5', u'P13_3', u'P13_4', u'P13_5', u'P14_0', u'P15_0', u'P16_0', u'P17_0', u'P18_0', u'P19_1', u'P19_2', u'P19_3', u'P19_4', u'P19_5', u'P20_1', u'P20_2', u'P20_3', u'P20_4', u'P20_5', u'P21_1', u'P21_2', u'P21_3', u'P21_4', u'P21_5', u'P22_1', u'P22_2', u'P22_3', u'P22_4', u'P22_5', u'P23_1', u'P23_2', u'P23_3', u'P23_4', u'P23_5', u'P24_0', u'P25_0', u'P26_0', u'P27_0', u'P28_1',

In [36]:
for name in names:
    print name
    print dict(test_normal[name].value_counts())
    print dict(train_normal[name].value_counts())

P1
{1: 5197, 2: 26562, 3: 25335, 4: 35147, 5: 7759}
{1.0: 10, 2.0: 42, 3.0: 33, 4.0: 47, 5.0: 4}
P2
{1.0: 1503, 2.0: 8719, 3.0: 17052, 4.0: 23009, 5.0: 49717}
{1.0: 5, 2.0: 13, 3.0: 16, 4.0: 39, 5.0: 63}
P3
{0.0: 318, 2.0: 1906, 3.0: 9959, 4.0: 77983, 5.0: 9834}
{0.0: 1, 2.0: 3, 3.0: 11, 4.0: 103, 5.0: 18}
P4
{2.0: 311, 3.0: 12376, 4.0: 66497, 5.0: 20816}
{3.0: 22, 4.0: 88, 5.0: 26}
P5
{1: 33587, 2: 50633, 3: 5931, 4: 9562, 5: 287}
{1.0: 59, 2.0: 64, 3.0: 7, 4.0: 5, 5.0: 1}
P6
{1: 9850, 2: 43925, 3: 20554, 4: 16873, 5: 8798}
{1.0: 12, 2.0: 61, 3.0: 28, 4.0: 17, 5.0: 18}
P7
{1: 6886, 2: 2223, 3: 1864, 4: 1884, 5: 87143}
{1.0: 12, 3.0: 1, 4.0: 3, 5.0: 120}
P8
{1: 639, 2: 1567, 3: 7767, 4: 44714, 5: 45313}
{1.0: 1, 2.0: 1, 3.0: 7, 4.0: 59, 5.0: 68}
P9
{3: 307, 4: 41681, 5: 58012}
{4.0: 46, 5.0: 90}
P10
{4: 25209, 5: 74791}
{4.0: 40, 5.0: 96}
P11
{1: 17781, 2: 23079, 3: 24118, 4: 17188, 5: 17834}
{1.0: 22, 2.0: 46, 3.0: 29, 4.0: 18, 5.0: 21}
P12
{2: 1275, 3: 11680, 4: 29330, 5: 57715}
{2.0

In [5]:
# freq of zeros
all_zeros = []

for i in range(0,len(test)):
    zero_c = test.iloc[i].value_counts()[0]
    if 0 not in test.iloc[i].value_counts().keys():
        zero_c = 0
    all_zeros.append(zero_c)

test["zeros"] = pd.Series(all_zeros)

print test.keys()

all_zeros_tr = []

for i in range(0,len(train)):
    zero_c = train.iloc[i].value_counts()[0]
    if 0 not in train.iloc[i].value_counts().keys():
        zero_c = 0
    all_zeros_tr.append(zero_c)

train["zeros"] = pd.Series(all_zeros_tr)

Index([u'Open Date', u'City', u'City Group', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'DaysOfRest', u'zeros'], dtype='object')


In [37]:
print test["zeros"].value_counts()
print train["zeros"].value_counts()

17    34207
0      7586
16     5662
15     4753
14     4200
1      3937
13     3842
12     3613
2      3377
11     3340
10     3178
9      3108
7      3068
3      3029
5      3003
4      3002
6      2992
8      2979
18     1124
dtype: int64
17    85
0     48
18     3
1      1
dtype: int64


In [10]:
# add max param feature and split for 2 parts
names = [name for name in train.keys() if name[0]=='P']
max_params = []
j = 0
for i in range(0,len(train)):
    max_value = 0
    for name in names:
        if train.iloc[i][name] > max_value:
            max_value = train.iloc[i][name]
    if max_value > 5:
        j +=1
    #print max_value
    max_params.append(max_value)
    
train['max_params'] = pd.Series(max_params)
big_params = train[train['max_params'] > 5]
small_params = train[train['max_params'] <= 5.0]
print len(big_params)
print len(small_params)



23
114


In [11]:
# convert strange data to good
names = [name for name in train.keys() if name[0]=='P']
multiplier = {'P1':3,'P2':1.5,'P3':1.5,'P4':1.5,'P5':2,'P6':2,'P7':2,'P8':2,'P9':2,'P10':2,'P11':2,'P12':2,'P13':1.5,'P14':3,
              'P15':2,'P16':3,'P17':3,'P18':3,'P19':5,'P20':3,'P21':3,'P22':1,'P23':5,'P24':2,'P25':2,'P26':2.5,'P27':2.5,
              'P28':2.5,'P29':2.5,'P30':5,'P31':3,'P32':5,'P33':2,'P34':6,'P35':3,'P36':4,'P37':2}

for i in range(0,len(train)):
    if train.iloc[i]['max_params'] > 5:
        for name in names:
            train.loc[i,name] /= multiplier[name]   


In [54]:
sum_arr = []
mul_arr = []
avg_arr = []
null_keys = ['P14', 'P15', 'P16', 'P17', 'P18', 'P24', 'P25', 
             'P26', 'P27', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37']
not_null_keys = list(set(names) - set(null_keys))
print not_null_keys
for i in range(0,len(train)):
    sums = 0
    mult = 1
    j = 1
    for name in not_null_keys:
        val = train.iloc[i][name]
        sums += val
        if val!=0:
            mult *= val
            j+=1
        
    sum_arr.append(sums)
    mul_arr.append(np.log(mult))
    avg_arr.append(sums/j)

train["sums"] = pd.Series(sum_arr)
train["avg"] = pd.Series(avg_arr)
train["log_mul"] = pd.Series(mul_arr)
print train.keys()

['P2', 'P3', 'P1', 'P6', 'P7', 'P4', 'P5', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P21', 'P20', 'P23', 'P22', 'P19', 'P29', 'P28']
Index([u'Open Date', u'City', u'City Group', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'revenue', u'DaysOfRest', u'zeros', u'max_params', u'sums', u'log_mul', u'avg'], dtype='object')


In [55]:
fig, axes = plt.subplots(nrows=1, ncols=3)
m = axes.flat
i=0
for name in ["sums","log_mul","avg"]:
    m[i].scatter(train["revenue"],train[name], alpha=0.5)
    i+=1

plt.show()

In [58]:
strange =  train[train["revenue"] > 10000000]
not_strange = train[train["revenue"] < 10000000]
for name in train.keys()[1:]:
    print name
    print dict(strange[name].value_counts())
    print dict(not_strange[name].value_counts())

City
{'\xc4\xb0stanbul': 3}
{'\xc4\xb0zmir': 9, 'Mu\xc4\x9fla': 2, 'Tekirda\xc4\x9f': 3, 'Trabzon': 2, 'Afyonkarahisar': 1, 'Adana': 3, 'K\xc4\xb1rklareli': 1, '\xc4\xb0stanbul': 47, 'Ayd\xc4\xb1n': 2, 'Gaziantep': 1, 'Ankara': 19, 'Kayseri': 3, 'Edirne': 1, 'Tokat': 1, 'Elaz\xc4\xb1\xc4\x9f': 1, 'Eski\xc5\x9fehir': 3, 'Bolu': 1, '\xc5\x9eanl\xc4\xb1urfa': 1, 'K\xc3\xbctahya': 1, 'Kocaeli': 1, 'Samsun': 5, 'U\xc5\x9fak': 1, 'Kastamonu': 1, 'Karab\xc3\xbck': 1, 'Isparta': 1, 'Denizli': 1, 'Diyarbak\xc4\xb1r': 3, 'Osmaniye': 1, 'Amasya': 1, 'Bal\xc4\xb1kesir': 1, 'Bursa': 5, 'Sakarya': 4, 'Antalya': 4, 'Konya': 2}
City Group
{'Big Cities': 3}
{'Big Cities': 75, 'Other': 59}
Type
{'FC': 2, 'IL': 1}
{'DT': 1, 'FC': 74, 'IL': 59}
P1
{3: 1, 4: 1, 5: 1}
{1: 10, 2: 42, 3: 33, 4: 46, 5: 3}
P2
{5.0: 3}
{1.0: 5, 2.0: 13, 3.0: 16, 4.0: 39, 5.0: 61}
P3
{4.0: 3}
{0.0: 1, 2.0: 3, 3.0: 11, 4.0: 101, 5.0: 18}
P4
{4.0: 2, 5.0: 1}
{3.0: 22, 4.0: 87, 5.0: 25}
P5
{1: 1, 2: 2}
{1: 58, 2: 63, 3: 7, 4: 5, 5: 

In [11]:
# get stat by type amount of zeros
names = train.keys()
names = names.drop("revenue")
print names
stat_names = list(names[2:])
stat_names.extend(["count"])
print stat_names
stats_data_test = pd.DataFrame(columns=stat_names)
zeros_amount_test = list(test["zeros"].value_counts().keys())
i = 0
for value in zeros_amount_test:
    row = pd.Series()
    bulk = test[test["zeros"] == value]
    for name in stats_data_test.keys()[:-2]:
        stat = dict(bulk[name].value_counts())
        row[name] = stat
        
    row["zeros"] = value
    row["count"] = len(bulk)
    stats_data_test.loc[i]= row.values
    i+=1
    

Index([u'Id', u'Open Date', u'City', u'City Group', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'DaysOfRest', u'zeros'], dtype='object')
['City', 'City Group', 'Type', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37', 'DaysOfRest', 'zeros', 'count']


In [12]:
# collect strange data
good_ids = list(not_null_data["Id"].values)
good_ids.extend(list(null_data["Id"].values))
cheat_ids = list(set(test["Id"]) - set(good_ids))
strange_data = test.iloc[cheat_ids]
for name in strange_data.keys()[3:]:
    print name
    print dict(strange_data[name].value_counts())

NameError: name 'not_null_data' is not defined

In [199]:
zero_split = strange_data[strange_data["zeros"] == 3]
for name in zero_split.keys()[3:]:
    print name
    print dict(zero_split[name].value_counts())


City Group
{'Big Cities': 1577, 'Other': 1452}
Type
{'DT': 57, 'FC': 647, 'IL': 2322, 'MB': 3}
P1
{1: 99, 2: 627, 3: 636, 4: 970, 5: 210, 6: 129, 9: 75, 12: 215, 15: 68}
P2
{7.5: 305, 4.5: 62, 2.0: 207, 3.0: 423, 4.0: 424, 5.0: 1450, 6.0: 114, 1.0: 41, 1.5: 3}
P3
{4.5: 86, 0.0: 3, 2.0: 20, 3.0: 136, 4.0: 2000, 5.0: 395, 6.0: 389}
P4
{7.5: 159, 4.5: 3, 3.0: 365, 4.0: 1665, 5.0: 510, 6.0: 327}
P5
{1: 1070, 2: 1437, 3: 117, 4: 329, 5: 3, 6: 73}
P6
{1: 205, 2: 1297, 3: 548, 4: 569, 5: 274, 6: 20, 8: 67, 10: 49}
P7
{1: 178, 2: 18, 3: 43, 4: 78, 5: 2232, 6: 3, 10: 477}
P8
{1: 22, 2: 92, 3: 242, 4: 1206, 5: 1044, 6: 59, 8: 187, 10: 177}
P9
{8: 232, 10: 245, 4: 1062, 5: 1473, 6: 17}
P10
{8: 149, 10: 307, 4: 695, 5: 1878}
P11
{1: 419, 2: 677, 3: 603, 4: 499, 5: 515, 6: 90, 8: 9, 10: 217}
P12
{2: 83, 3: 408, 4: 633, 5: 1475, 6: 108, 8: 94, 10: 228}
P13
{7.5: 332, 3.0: 50, 4.0: 646, 5.0: 1860, 6.0: 141}
P14
{0: 540, 1: 374, 2: 358, 3: 1049, 4: 204, 5: 171, 6: 86, 9: 113, 12: 37, 15: 97}
P15
{0: 5

In [29]:
for name in big_params.keys()[3:]:
    print name
    print dict(big_params[name].value_counts())
    print dict(small_params[name].value_counts())

City Group
{'Big Cities': 15, 'Other': 8}
{'Big Cities': 63, 'Other': 51}
Type
{'FC': 9, 'IL': 14}
{'DT': 1, 'FC': 67, 'IL': 46}
P1
{9: 4, 3: 1, 12: 12, 6: 6}
{1: 9, 2: 36, 3: 30, 4: 35, 5: 4}
P2
{7.5: 13, 4.5: 3, 3.0: 2, 6.0: 5}
{1.0: 5, 2.0: 11, 3.0: 13, 4.0: 34, 5.0: 51}
P3
{7.5: 2, 4.5: 1, 6.0: 20}
{0.0: 1, 2.0: 3, 3.0: 10, 4.0: 84, 5.0: 16}
P4
{7.5: 4, 4.5: 2, 6.0: 17}
{3.0: 20, 4.0: 72, 5.0: 22}
P5
{8: 2, 2: 11, 4: 9, 6: 1}
{1: 48, 2: 56, 3: 6, 4: 3, 5: 1}
P6
{8: 7, 10: 5, 4: 5, 2: 1, 6: 5}
{1: 11, 2: 56, 3: 23, 4: 10, 5: 14}
P7
{10: 22, 2: 1}
{1: 11, 3: 1, 4: 3, 5: 99}
P8
{8: 11, 10: 11, 6: 1}
{1: 1, 2: 1, 3: 6, 4: 49, 5: 57}
P9
{8: 8, 10: 15}
{4: 38, 5: 76}
P10
{8: 7, 10: 16}
{4: 34, 5: 80}
P11
{8: 4, 10: 3, 4: 9, 2: 2, 6: 5}
{1: 20, 2: 37, 3: 25, 4: 14, 5: 18}
P12
{8: 6, 10: 15, 6: 2}
{2: 4, 3: 8, 4: 26, 5: 76}
P13
{7.5: 16, 6.0: 7}
{3.0: 2, 4.0: 32, 5.0: 80}
P14
{0: 11, 3: 3, 6: 3, 9: 3, 12: 1, 15: 2}
{0: 77, 1: 9, 2: 7, 3: 12, 4: 6, 5: 3}
P15
{0: 11, 8: 3, 10: 4, 4: 4, 2: 1}

In [43]:
for name in big_params.keys()[3:]:
    print name
    print big_params[name].value_counts()

City Group
Big Cities    15
Other          8
dtype: int64
Type
IL    14
FC     9
dtype: int64
P1
4    12
2     6
3     4
1     1
dtype: int64
P2
5    13
4     5
3     3
2     2
dtype: int64
P3
4    20
5     2
3     1
dtype: int64
P4
4    17
5     4
3     2
dtype: int64
P5
1    11
2     9
4     2
3     1
dtype: int64
P6
4    7
5    5
3    5
2    5
1    1
dtype: int64
P7
5    22
1     1
dtype: int64
P8
5    11
4    11
3     1
dtype: int64
P9
5    15
4     8
dtype: int64
P10
5    16
4     7
dtype: int64
P11
2    9
3    5
4    4
5    3
1    2
dtype: int64
P12
5    15
4     6
3     2
dtype: int64
P13
5    16
4     7
dtype: int64
P14
0    11
3     3
2     3
1     3
5     2
4     1
dtype: int64
P15
0    11
5     4
2     4
4     3
1     1
dtype: int64
P16
0    11
4     5
3     4
5     3
dtype: int64
P17
0    11
1     8
2     2
5     1
3     1
dtype: int64
P18
0    11
4     9
3     2
1     1
dtype: int64
P19
5    6
2    6
3    5
1    4
4    2
dtype: int64
P20
5    9
4    5
3    5
1    3
2    1


In [244]:
# add max param feature and split for 2 parts for test
big_params_test = test[test['max_params'] > 5]
small_params_test = test[test['max_params'] <= 5.0]
print len(big_params_test)
print len(small_params_test)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
33901
66099


In [245]:
for name in big_params_test.keys()[3:]:
    print name
    print dict(big_params_test[name].value_counts())
    print dict(small_params_test[name].value_counts())

City Group
{'Big Cities': 17012, 'Other': 16889}
{'Big Cities': 32260, 'Other': 33839}
Type
{'DT': 495, 'FC': 18655, 'IL': 14690, 'MB': 61}
{'DT': 1749, 'FC': 38364, 'IL': 25757, 'MB': 229}
P1
{1: 1178, 2: 5046, 3: 5497, 4: 6260, 5: 1589, 6: 4092, 9: 2490, 12: 6852, 15: 897}
{1: 4019, 2: 17424, 3: 17348, 4: 22035, 5: 5273}
P2
{7.5: 8457, 4.5: 2235, 2.0: 2000, 3.0: 3555, 4.0: 4311, 5.0: 9095, 6.0: 3689, 1.0: 264, 1.5: 295}
{4.5: 39, 1.0: 921, 2.0: 6719, 3.0: 11223, 4.0: 15009, 5.0: 32165, 1.5: 23}
P3
{4.5: 2732, 0.0: 74, 2.0: 397, 3.0: 2323, 4.0: 14927, 5.0: 2163, 6.0: 11285}
{0.0: 244, 4.5: 49, 2.0: 1509, 3.0: 4855, 4.0: 51771, 5.0: 7671}
P4
{7.5: 4359, 4.5: 575, 2.0: 74, 3.0: 2695, 4.0: 12556, 5.0: 3653, 6.0: 9989}
{4.5: 31, 2.0: 237, 3.0: 9075, 4.0: 43952, 5.0: 12804}
P5
{1: 7573, 2: 14302, 3: 822, 4: 8929, 5: 65, 6: 2210}
{1: 26014, 2: 36331, 3: 2899, 4: 633, 5: 222}
P6
{1: 2201, 2: 11288, 3: 4017, 4: 10045, 5: 1707, 6: 2474, 8: 921, 10: 1248}
{1: 7649, 2: 32637, 3: 14063, 4: 5907, 

In [249]:
# print train split by params
#print len(big_params)
#print len(small_params)
fig, axes = plt.subplots(nrows=7, ncols=6)
m = axes.flat
for i in range(5,len(names)):
    m[i-5].scatter(small_params[names[i]],small_params["revenue"]/1000000, alpha=0.5)



In [258]:
for name in big_params_test.keys()[3:]:
    print name
    print dict(some_split[name].value_counts())

City Group
{'Big Cities': 255, 'Other': 201}
Type
{'DT': 2, 'FC': 146, 'IL': 308}
P1
{1: 2, 2: 16, 3: 29, 4: 33, 6: 85, 9: 4, 12: 198, 15: 89}
P2
{7.5: 191, 4.5: 4, 3.0: 12, 4.0: 26, 5.0: 46, 6.0: 177}
P3
{4.5: 86, 0.0: 1, 2.0: 1, 3.0: 7, 4.0: 78, 5.0: 5, 6.0: 278}
P4
{7.5: 83, 2.0: 1, 3.0: 11, 4.0: 61, 5.0: 24, 6.0: 276}
P5
{2: 252, 3: 6, 4: 113, 6: 85}
P6
{1: 8, 2: 60, 3: 18, 4: 193, 5: 8, 6: 8, 8: 74, 10: 87}
P7
{1: 5, 10: 374, 3: 2, 4: 3, 5: 72}
P8
{1: 2, 2: 4, 3: 7, 4: 57, 5: 31, 6: 3, 8: 266, 10: 86}
P9
{8: 190, 10: 208, 4: 58}
P10
{8: 106, 10: 320, 4: 30}
P11
{1: 11, 2: 28, 3: 22, 4: 104, 5: 14, 6: 262, 8: 3, 10: 12}
P12
{2: 7, 3: 19, 4: 34, 5: 36, 6: 86, 8: 88, 10: 186}
P13
{7.5: 270, 3.0: 1, 4.0: 33, 5.0: 69, 6.0: 83}
P14
{1: 7, 2: 11, 3: 216, 4: 6, 5: 1, 6: 6, 9: 113, 12: 1, 15: 95}
P15
{0: 55, 1: 4, 2: 13, 3: 5, 4: 97, 5: 6, 6: 82, 8: 189, 10: 5}
P16
{0: 73, 2: 3, 3: 12, 4: 12, 5: 4, 6: 79, 12: 270, 15: 3}
P17
{0: 62, 1: 19, 2: 8, 3: 186, 5: 6, 9: 90, 12: 1, 15: 84}
P18
{0: 

In [261]:
fig, axes = plt.subplots(nrows=7, ncols=6)
m = axes.flat
for i in range(5,len(names)):
    m[i-5].hist(list(big_params_test[names[i]]), bins = 10)

plt.show()

In [ ]:
# rest days research
gr_d = dict(test["DaysOfRest"].value_counts())
plt.scatter(gr_d.keys(),gr_d.values(), alpha=0.5)
gr_d = dict(train["DaysOfRest"].value_counts())
plt.scatter(gr_d.keys(),gr_d.values(), alpha=0.5)

In [63]:
# test distribution
import random
days = []
for i in range(0,1200):
    days.append(random.randint(400,7300))
days = pd.Series(days)
print days.value_counts()
gr_d = dict(days.value_counts())
plt.scatter(gr_d.keys(),gr_d.values(), alpha=0.5)

1933    3
2067    3
2747    3
2417    3
4515    3
5990    2
5781    2
3319    2
4247    2
5414    2
6872    2
5160    2
1940    2
635     2
455     2
...
3280    1
5371    1
1542    1
5363    1
3314    1
5360    1
2700    1
3306    1
2873    1
3301    1
1247    1
5069    1
3269    1
1235    1
2049    1
Length: 1088, dtype: int64


In [12]:
# research linear shit
print multiplier
names = [name for name in train.keys() if name[0]=='P']
print names
lin_counts = []
prob_bads = []
for i in range(0,len(test)):
    lin_count = 0
    prob_bad = 0
    for name in names:
        val = test.iloc[i][name]
        if val % 1 == 0 and val // 1 <= 5:
            if val % multiplier[name] == 0 and val // multiplier[name] > 0 and multiplier[name]!=1:
                prob_bad+=1 
        else:
            lin_count += 1
    if i % 5000 == 0:
        print i
    lin_counts.append(lin_count)
    prob_bads.append(prob_bad)
    #if train.iloc[i]["max_params"] > 5:
     #   print i
#3 % 1 == 0; 3 % 1.5 == 0 7.5 % 1 <=5
    

{'P2': 1.5, 'P3': 1.5, 'P1': 3, 'P6': 2, 'P7': 2, 'P4': 1.5, 'P5': 2, 'P35': 3, 'P8': 2, 'P9': 2, 'P22': 1, 'P32': 5, 'P26': 2.5, 'P23': 5, 'P33': 2, 'P25': 2, 'P24': 2, 'P36': 4, 'P21': 3, 'P27': 2.5, 'P37': 2, 'P10': 2, 'P11': 2, 'P12': 2, 'P13': 1.5, 'P14': 3, 'P15': 2, 'P16': 3, 'P17': 3, 'P18': 3, 'P19': 5, 'P30': 5, 'P31': 3, 'P29': 2.5, 'P28': 2.5, 'P34': 6, 'P20': 3}
['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37']
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000


In [13]:
lin_counts = pd.Series(lin_counts)
print lin_counts.value_counts()
prob_bads = pd.Series(prob_bads)
print prob_bads.value_counts()
test['linear_counts'] = lin_counts
test['prob_bads'] = prob_bads

0     65871
1      3565
2      3113
3      2746
4      2408
5      2167
6      1950
7      1783
8      1758
9      1573
10     1488
11     1393
12     1310
13     1264
14     1095
15     1039
16      845
17      700
18      485
19      407
20      406
21      369
22      363
23      342
24      301
25      278
26      264
27      218
28      172
29      119
30       94
31       69
32       45
dtype: int64
5     13195
6     13146
4     12831
7     11228
3     10529
8      9169
9      6804
2      6782
10     5011
11     3619
1      2736
12     2246
13     1240
14      640
0       508
15      208
16       86
17       18
18        3
19        1
dtype: int64


In [16]:
test.to_csv("raw/test_with_zeros_lin_bad.csv", index=False)

In [39]:
# add random permutations
import random
numeric_names = [name for name in names if name[0]=='P']
multiplier = {'P1':3,'P2':1.5,'P3':1.5,'P4':1.5,'P5':2,'P6':2,'P7':2,'P8':2,'P9':2,'P10':2,
              'P11':2,'P12':2,'P13':1.5,'P14':3,'P15':2,'P16':3,'P17':3,'P18':3,'P19':5,
              'P20':3,'P21':3,'P22':1,'P23':5,'P24':2,'P25':2,'P26':2.5,'P27':2.5, 'P28':2.5,
              'P29':2.5,'P30':5,'P31':3,'P32':5,'P33':2,'P34':6,'P35':3,'P36':4,'P37':2}   

print test["linear_counts"].value_counts().keys()
print test["zeros"].value_counts().keys()
train_zeros = dict(train["zeros"].value_counts())
print train_zeros
for key in train_zeros.keys():
        short_slice = test[test["zeros"]==key]
        print key
        print dict(((short_slice["linear_counts"].value_counts()/len(short_slice["linear_counts"]))*train_zeros[key]).astype(np.int64))
for key in train_zeros.keys():
    print "Zeros count: ", str(key)
    short_slice = test[test["zeros"]==key]
    for name in numeric_names:
        print name
        print dict(short_slice[name].value_counts())

# train
#17    85
#0     48
#18     3
#1      1

# test distrib params to tweak
# 0
#{1: 5, 2: 3, 3: 2, 4: 2, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 
#11: 1, 12: 1, 13: 1, 16: 1, 18: 1,
#21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1}
# 17
#{0: 22, 1: 8, 2: 6, 3: 5, 4: 4, 5: 4, 6: 3, 7: 3, 8: 3, 9: 2, 
#10: 2, 11: 3, 12: 2, 13: 2, 14: 2, 15: 2, 16: 1, 17: 1}
#
# P22 dont touch

#0     65871 - 2/3 normal values
#1      3565
#2      3113
#3      2746
#4      2408
#5      2167
#6      1950
#7      1783
#8      1758
#9      1573
#10     1488
#11     1393
#12     1310
#13     1264
#14     1095
#15     1039
#16      845
#17      700
#18      485
#19      407
#20      406
#21      369
#22      363
#23      342
#24      301
#25      278
#26      264
#27      218
#28      172
#29      119
#30       94
#31       69
#32       45

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32], dtype='int64')
Int64Index([18, 17, 1, 16, 15, 14, 2, 13, 12, 3, 11, 10, 9, 4, 5, 7, 8, 6, 0, 19, 20], dtype='int64')
{0: 48, 17: 85, 18: 3, 1: 1}
0
{1: 5, 2: 3, 3: 2, 4: 2, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 0, 15: 0, 16: 1, 17: 0, 18: 1, 19: 0, 20: 0, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 0, 32: 0}
17
{0: 22, 1: 8, 2: 6, 3: 5, 4: 4, 5: 4, 6: 3, 7: 3, 8: 3, 9: 2, 10: 2, 11: 3, 12: 2, 13: 2, 14: 2, 15: 2, 16: 1, 17: 1, 18: 0}
18
{0: 2, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 19: 0}
1
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0}
Zeros count:  0
P1
{1:

In [40]:
print  test["zeros"].value_counts()

18    23799
17    14054
1      6227
16     4997
15     4310
14     3945
2      3842
13     3758
12     3438
3      3257
11     3206
10     3166
9      3074
4      3073
5      3056
7      3051
8      2973
6      2945
0      2631
19     1187
20       11
dtype: int64


In [47]:
print train["City Group"].value_counts()
# İstanbul - 1
# Ankara - 2
# İzmir - 3
# Other - 4
print test[test["City Group"]=="Other"]["City"].value_counts()

Big Cities    78
Other         59
dtype: int64
Antalya           5911
Kocaeli           4364
Mersin            2735
Adana             2514
Balıkesir         2463
Bursa             2441
Muğla             1823
Aydın             1617
Tekirdağ          1577
Konya             1576
Gaziantep         1487
Edirne            1230
Manisa            1227
Çanakkale          965
Denizli            964
Diyarbakır         954
Hatay              951
Zonguldak          926
Eskişehir          900
Trabzon            660
Aksaray            650
Bolu               631
Yalova             630
Kırıkkale          622
Malatya            616
Mardin             610
Şanlıurfa          609
Sakarya            604
Batman             604
Rize               345
Artvin             344
Bilecik            339
Afyonkarahisar     331
Nevşehir           328
Sivas              326
Samsun             324
Kayseri            323
Kırşehir           319
Erzincan           319
Ordu               317
Erzurum            317
Siirt     

In [49]:
# replace low freq city
big_city = ['İstanbul', 'Ankara', 'İzmir']
for i in range(0,len(train)):
    if train.iloc[i]["City"] not in big_city:
        train.loc[i,"City"] = 'Other'

In [50]:
train["City"].value_counts()

Other       59
İstanbul    50
Ankara      19
İzmir        9
dtype: int64

In [54]:
# fill missed values of type
from random import randint
import random
random.seed( 10 )

pop_types = ["FC","IL"]
for i in range(0,len(train)):
    if train.iloc[i]["Type"] not in pop_types:
        train.loc[i,"Type"] = pop_types[randint(0,len(pop_types))-1]

In [55]:
print train["Type"].value_counts()

FC    76
IL    61
dtype: int64


In [61]:
print multiplier
test_normal = test.copy()
names = [name for name in train.keys() if name[0]=='P']
print names
for i in range(0,len(test)):
    for name in names:
        val = test_normal.iloc[i][name]
        if val % 1 != 0 or val > 5:
            test_normal.loc[i,name] /= multiplier[name]
    if i % 5000 == 0:
        print i

{'P2': 1.5, 'P3': 1.5, 'P1': 3, 'P6': 2, 'P7': 2, 'P4': 1.5, 'P5': 2, 'P35': 3, 'P8': 2, 'P9': 2, 'P22': 1, 'P32': 5, 'P26': 2.5, 'P23': 5, 'P33': 2, 'P25': 2, 'P24': 2, 'P36': 4, 'P21': 3, 'P27': 2.5, 'P37': 2, 'P10': 2, 'P11': 2, 'P12': 2, 'P13': 1.5, 'P14': 3, 'P15': 2, 'P16': 3, 'P17': 3, 'P18': 3, 'P19': 5, 'P30': 5, 'P31': 3, 'P29': 2.5, 'P28': 2.5, 'P34': 6, 'P20': 3}
['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37']
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000


In [63]:
has_zeros = []
missed_values = ['P4','P7','P9','P18','P29', 'P34', 'P36']
for name in names:
    print name
    print test_normal[name].value_counts().keys()
    if 0 in test_normal[name].value_counts().get_values():
        has_zeros.append(name)

# 3, 4, 7,9,10,12,13,18,29,34,35,36

P1
Int64Index([4, 2, 3, 5, 1], dtype='int64')
P2
Float64Index([5.0, 4.0, 3.0, 2.0, 1.0], dtype='float64')
P3
Float64Index([4.0, 3.0, 5.0, 2.0, 0.0], dtype='float64')
P4
Float64Index([4.0, 5.0, 3.0, 2.0], dtype='float64')
P5
Int64Index([2, 1, 4, 3, 5], dtype='int64')
P6
Int64Index([2, 3, 4, 1, 5], dtype='int64')
P7
Int64Index([5, 1, 2, 4, 3], dtype='int64')
P8
Int64Index([5, 4, 3, 2, 1], dtype='int64')
P9
Int64Index([5, 4, 3], dtype='int64')
P10
Int64Index([5, 4], dtype='int64')
P11
Int64Index([3, 2, 5, 1, 4], dtype='int64')
P12
Int64Index([5, 4, 3, 2], dtype='int64')
P13
Float64Index([5.0, 4.0, 3.0], dtype='float64')
P14
Int64Index([0, 3, 2, 1, 5, 4], dtype='int64')
P15
Int64Index([0, 4, 2, 5, 3, 1], dtype='int64')
P16
Int64Index([0, 4, 5, 3, 2, 1], dtype='int64')
P17
Int64Index([0, 3, 1, 2, 5, 4], dtype='int64')
P18
Int64Index([0, 4, 3, 5, 1, 2], dtype='int64')
P19
Int64Index([5, 3, 2, 4, 1], dtype='int64')
P20
Int64Index([5, 1, 4, 3, 2], dtype='int64')
P21
Int64Index([1, 2, 3, 5, 4],

In [67]:
missed_values = ['P4','P7','P9','P18','P29', 'P34', 'P36']
for name in missed_values:
    print name, multiplier[name]
    print test_normal[name].value_counts()

P4 1.5
4    66497
5    20816
3    12376
2      311
dtype: int64
P7 2
5    87143
1     6886
2     2223
4     1884
3     1864
dtype: int64
P9 2
5    58012
4    41681
3      307
dtype: int64
P18 3
0    65980
4    19619
3     7211
5     4941
1     1927
2      322
dtype: int64
P29 2.5
3    65381
2    17339
1    10781
5     3112
0     3083
4      304
dtype: int64
P34 6
0    65832
3    13963
4    12163
5     4229
2     2863
1      950
dtype: int64
P36 4
0    65662
4    13963
3    10693
5     7805
2     1564
1      313
dtype: int64
